In [1]:
# In this exercise you will train a CNN on the FULL Cats-v-dogs dataset
# This will require you doing a lot of data preprocessing because
# the dataset isn't split into training and validation for you
# This code block has all the required inputs
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from numpy.random import uniform

/home/beltain/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/beltain/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/beltain/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/beltain/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [3]:
# This code block downloads the full Cats-v-Dogs dataset and stores it as 
# cats-and-dogs.zip. It then unzips it to /tmp
# which will create a tmp/PetImages directory containing subdirectories
# called 'Cat' and 'Dog' (that's how the original researchers structured it)
# If the URL doesn't work, 
# .   visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL
if True:
    !wget --no-check-certificate \
        "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
        -O "../../Data/cats-and-dogs.zip"

    local_zip = '../../Data/cats-and-dogs.zip'
else:
    local_zip = '../../Data/cats_and_dogs_filtered.zip'
    
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('../../Data/tmp')
zip_ref.close()


--2019-12-24 13:25:44--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.45.144.230, 2600:1408:8400:3a9::e59, 2600:1408:8400:385::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.45.144.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘../../Data/cats-and-dogs.zip’

../../Data/cats-and 100%[===================>] 786.68M  9.13MB/s    in 58s     

2019-12-24 13:26:42 (13.6 MB/s) - ‘../../Data/cats-and-dogs.zip’ saved [824894548/824894548]



In [4]:
print(len(os.listdir('../../Data/tmp/PetImages/Cat/')))
print(len(os.listdir('../../Data/tmp/PetImages/Dog/')))

# Expected Output:
# 12501
# 12501

12501
12501


In [22]:
# Use os.mkdir to create your directories
# You will need a directory for cats-v-dogs, and subdirectories for training
# and testing. These in turn will need subdirectories for 'cats' and 'dogs'
try:
    os.mkdir('../../Data/tmp/cats-v-dogs')
    os.mkdir('../../Data/tmp/cats-v-dogs/training')
    os.mkdir('../../Data/tmp/cats-v-dogs/testing')
    os.mkdir('../../Data/tmp/cats-v-dogs/training/cats')
    os.mkdir('../../Data/tmp/cats-v-dogs/training/dogs')
    os.mkdir('../../Data/tmp/cats-v-dogs/testing/cats')
    os.mkdir('../../Data/tmp/cats-v-dogs/testing/dogs')
except OSError:
    print('no go jo')

In [23]:
# Write a python function called split_data which takes
# a SOURCE directory containing the files
# a TRAINING directory that a portion of the files will be copied to
# a TESTING directory that a portion of the files will be copie to
# a SPLIT SIZE to determine the portion
# The files should also be randomized, so that the training set is a random
# X% of the files, and the test set is the remaining files
# SO, for example, if SOURCE is PetImages/Cat, and SPLIT SIZE is .9
# Then 90% of the images in PetImages/Cat will be copied to the TRAINING dir
# and 10% of the images will be copied to the TESTING dir
# Also -- All images should be checked, and if they have a zero file length,
# they will not be copied over
#
# os.listdir(DIRECTORY) gives you a listing of the contents of that directory
# os.path.getsize(PATH) gives you the size of the file
# copyfile(source, destination) copies a file from source to destination
# random.sample(list, len(list)) shuffles a list
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    TRAINING_C = 0
    TESTING_C = 0
    for file_idx in  os.listdir(SOURCE):
        if os.path.getsize(SOURCE + file_idx) > 1:
            if uniform() <= SPLIT_SIZE:
                copyfile(SOURCE + file_idx, TRAINING + file_idx)
                TRAINING_C += 1
            else:
                copyfile(SOURCE + file_idx, TESTING + file_idx)
                TESTING_C += 1
        else:
            print("{} is of size 0".format(file_idx))
            
    print('Number of Test: {}\nNumber of Training: {}\nRation: {}'.format(TESTING_C, TRAINING_C, TESTING_C/(TRAINING_C+TESTING_C)))
    
    

if False:
    CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
    DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
    TRAINING_DIR = "/tmp/cats-v-dogs/training/"
    TESTING_DIR = "/tmp/cats-v-dogs/testing/"

else:
    CAT_SOURCE_DIR = "../../Data/tmp/PetImages/Cat/"
    DOG_SOURCE_DIR = "../../Data/tmp/PetImages/Dog/"
    TRAINING_DIR = "../../Data/tmp/cats-v-dogs/training/"
    TESTING_DIR = "../../Data/tmp/cats-v-dogs/testing/"
 

    TRAINING_CATS_DIR = TRAINING_DIR + 'cats/'
    TESTING_CATS_DIR =  TESTING_DIR + 'cats/'

    TRAINING_DOGS_DIR = TRAINING_DIR + 'dogs/'
    TESTING_DOGS_DIR = TESTING_DIR + 'dogs/'

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

666.jpg is of size 0
Number of Test: 1228
Number of Training: 11272
Ration: 0.09824
11702.jpg is of size 0
Number of Test: 1219
Number of Training: 11281
Ration: 0.09752


In [24]:
if False:
    print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
    print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
    print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
    print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))
else:
    print(len(os.listdir('../../Data/tmp/cats-v-dogs/training/cats/')))
    print(len(os.listdir('../../Data/tmp/cats-v-dogs/training/dogs/')))
    print(len(os.listdir('../../Data/tmp/cats-v-dogs/testing/cats/')))
    print(len(os.listdir('../../Data/tmp/cats-v-dogs/testing/dogs/')))

# Expected output:
# 11250
# 11250
# 1250
# 1250

11272
11281
1228
1219


In [25]:
# DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
# USE AT LEAST 3 CONVOLUTION LAYERS
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (150,150,3) ),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation = 'relu' ),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [26]:
#RAINING_DIR = '../../Data/'
train_datagen = ImageDataGenerator(rescale = 1/255.)

train_generator = train_datagen.flow_from_directory(
                        TRAINING_DIR,
                        batch_size = 100,
                        class_mode= 'binary',
                        target_size = (150,150))

#VALIDATION_DIR = #YOUR CODE HERE
test_datagen = ImageDataGenerator(rescale = 1/255.)
test_generator = test_datagen.flow_from_directory(
                        TESTING_DIR,
                        batch_size = 100,
                        class_mode = 'binary',
                        target_size = (150,150)
)



# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

Found 22551 images belonging to 2 classes.
Found 2447 images belonging to 2 classes.


In [27]:
history = model.fit_generator(train_generator,
                              epochs=15,
                              verbose=1,
                              validation_data=test_generator)

# The expectation here is that the model will train, and that accuracy will be > 95% on both training and validation
# i.e. acc:A1 and val_acc:A2 will be visible, and both A1 and A2 will be > .9

Epoch 1/15
225/226 [============================>.] - ETA: 0s - loss: 0.6369 - acc: 0.6583

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

226/226 [==============================] - 36s 161ms/step - loss: 0.6362 - acc: 0.6587 - val_loss: 0.4796 - val_acc: 0.7740
Epoch 2/15
226/226 [==============================] - 34s 153ms/step - loss: 0.4862 - acc: 0.7668 - val_loss: 0.5713 - val_acc: 0.7323
Epoch 3/15
226/226 [==============================] - 35s 153ms/step - loss: 0.4152 - acc: 0.8087 - val_loss: 0.4497 - val_acc: 0.7785
Epoch 4/15
226/226 [==============================] - 34s 152ms/step - loss: 0.3537 - acc: 0.8413 - val_loss: 0.4061 - val_acc: 0.8190
Epoch 5/15
226/226 [==============================] - 35s 153ms/step - loss: 0.2829 - acc: 0.8778 - val_loss: 0.4249 - val_acc: 0.8169
Epoch 6/15
226/226 [==============================] - 35s 153ms/step - loss: 0.2168 - acc: 0.9104 - val_loss: 0.4658 - val_acc: 0.8186
Epoch 7/15
226/226 [==============================] - 34s 153ms/step - loss: 0.1485 - acc: 0.9417 - val_loss: 0.5293 - val_acc: 0.8186
Epoch 8/15
226/226 [==============================] - 35s 154ms/st

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

# Desired output. Charts with training and validation metrics. No crash :)

In [ ]:
# Here's a codeblock just for fun. You should be able to upload an image here 
# and have it classified without crashing

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(# YOUR CODE HERE))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")